In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

import catboost as cat
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit, RepeatedStratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import linear_model
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")


In [83]:
import json

In [84]:
specs = pd.read_csv('specs.csv')

In [85]:
specs.args=specs.args.apply(json.loads)

In [86]:
specs_exp=specs.explode('args')

In [87]:
specs_exp['name']=[i['name'] for i in [specs_exp.args.iloc[j] for j in range(specs_exp.shape[0])]]
specs_exp['type']=[i['type'] for i in [specs_exp.args.iloc[j] for j in range(specs_exp.shape[0])]]
specs_exp['info']=[i['info'] for i in [specs_exp.args.iloc[j] for j in range(specs_exp.shape[0])]]
specs_exp=specs_exp.drop('args',1)

In [111]:
specs_exp=specs_exp.loc[(specs_exp['name']!='event_count')]
specs_exp=specs_exp.loc[(specs_exp['name']!='event_code')]

In [112]:
specs_exp.describe()

,event_id,info,name,type
count,1384,1384,1384,1384
unique,381,319,119,8
top,5c2f29ca,number of the current round when the event tak...,round,int
freq,9,197,220,623


In [153]:
specs_exp=specs_exp.loc[(specs_exp['name']!='game_time')]
specs_exp=specs_exp.loc[(specs_exp['name']!='round')]
specs_exp=specs_exp.loc[(specs_exp['name']!='level')]
specs_exp=specs_exp.loc[(specs_exp['name']!='description')]
specs_exp=specs_exp.loc[(specs_exp['name']!='identifier')]

In [154]:
specs_exp=specs_exp.loc[(specs_exp['name']!='duration')]
specs_exp=specs_exp.loc[(specs_exp['name']!='media_type')]
specs_exp=specs_exp.loc[(specs_exp['name']!='total_duration')]

In [155]:
specs_exp=specs_exp.loc[(specs_exp['name']!='movie_id')]

In [156]:
specs_exp.describe()

,event_id,info,name,type
count,605,605,605,605
unique,246,290,111,8
top,0db6d71d,"dictionary: \n {""x"":integer, ""y"":integer, ""...",coordinates,int
freq,8,23,103,196


In [157]:
specs_exp.drop_duplicates(subset='info')

,event_id,info,name,type
6,bc8f2793,"dictionary: \n {""x"":integer, ""y"":integer, ""...",coordinates,object
6,bc8f2793,"""space"" or ""resource bin""",destination,string
6,bc8f2793,number of weights on the table after the event...,table_weights,int
6,bc8f2793,the new number of weights on the scale after t...,weights,int
8,cdd22e43,“chicken” or “pig” – what object was picked up,object,string
...,...,...,...,...
373,38074c54,the number of times the done button was presse...,misses,int
383,2a444e03,"""table"" or ""balance"" the location the weight w...",location,string
383,2a444e03,number of weights on the scale after the weigh...,scale_weights,int
383,2a444e03,number of weights on the table after the weigh...,table_weights,int


In [147]:
specs_exp.loc[specs_exp['name']=='object']

,event_id,info,name,type
8,cdd22e43,“chicken” or “pig” – what object was picked up,object,string
10,119b5b02,the object or piece of art the player clicked on,object,string
20,ecc6157f,the object or piece of art the player clicked on,object,string
34,7fd1ac25,the object or piece of art the player clicked on,object,string
43,5dc079d8,the object or piece of art the player clicked on,object,string
47,56bcd38d,“chicken” or “pig” – what object was picked up,object,string
57,26a5a3dd,the object the mouse was hovering over,object,string
73,9554a50b,the object or piece of art the player clicked on,object,string
114,16667cc5,the object or piece of art the player clicked on,object,string
141,6f445b57,the object or piece of art the player clicked on,object,string


In [106]:
i=300

In [159]:
specs.args.iloc[i]

[{'name': 'game_time',
  'type': 'int',
  'info': 'millisecond count since start of game'},
 {'name': 'side',
  'type': 'string',
  'info': '“left” or “right” – side the chicken was placed on'},
 {'name': 'layout',
  'type': 'object',
  'info': 'dictionary describing the state of the see-saw after the event occurs:\n{“left”: {“chickens”: int – count of chickens on the left side, “pig”: boolean – is pig on the left side?},\n“right”: {“chickens”: int – count of chickens on the right side, “pig”: boolean – is pig on the right side?}}'},
 {'name': 'coordinates',
  'type': 'object',
  'info': 'the game screen coordinates of the placement\ndictionary:\n{“x”:integer, “y”:integer, “stage_width”:integer, “stage_height”:integer}'},
 {'name': 'duration',
  'type': 'int',
  'info': 'the duration of the drag in milliseconds'},
 {'name': 'source',
  'type': 'string',
  'info': '“left”, “right”, or “resources” – location the chicken was dragged from'},
 {'name': 'event_count', 'type': 'int', 'info': 

In [160]:
specs.args.iloc[i+1]

[{'name': 'game_time',
  'type': 'int',
  'info': 'millisecond count since start of game'},
 {'name': 'growth',
  'type': 'array->int',
  'info': 'list of the changes in height for each flower from left to right'},
 {'name': 'flowers',
  'type': 'array->int',
  'info': 'list of the height of each flower from left to right'},
 {'name': 'duration',
  'type': 'int',
  'info': 'the duration of the drag in milliseconds'},
 {'name': 'event_count', 'type': 'int', 'info': 'session event counter'},
 {'name': 'event_code', 'type': 'int', 'info': 'event class identifier'}]

In [161]:
specs.args.iloc[i+2]

[{'name': 'game_time',
  'type': 'int',
  'info': 'millisecond count since start of game'},
 {'name': 'layout',
  'type': 'object',
  'info': 'dictionary describing the state of the gates after the event occurs with row 1 being the top row of gates and empty used to indicate no gate in that position:\n{“row1”:[“empty”, “left”, “empty”],\n“row2”:[“right”, “left”, “middle”], etc.}'},
 {'name': 'event_count', 'type': 'int', 'info': 'session event counter'},
 {'name': 'event_code', 'type': 'int', 'info': 'event class identifier'}]

In [162]:
specs.args.iloc[i+3]

[{'name': 'game_time',
  'type': 'int',
  'info': 'millisecond count since start of game'},
 {'name': 'description',
  'type': 'string',
  'info': 'the text or description of the instruction'},
 {'name': 'identifier',
  'type': 'string',
  'info': 'a unique identifier for this piece of instruction'},
 {'name': 'media_type',
  'type': 'string',
  'info': "the type of media that has just played:\n'audio' || 'animation' || 'other'"},
 {'name': 'duration',
  'type': 'int',
  'info': 'the duration of the media playback in milliseconds'},
 {'name': 'event_count', 'type': 'int', 'info': 'session event counter'},
 {'name': 'event_code', 'type': 'int', 'info': 'event class identifier'}]